## 语料库数据

In [1]:
dir='./data/shakespeare_input.txt'
with open(dir,'r',encoding='utf-8') as f:
    text=f.read()


In [2]:
print(len(text))#整个语料库大小

1115394


In [3]:
#here are the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'vocabulary size:{vocab_size}')



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocabulary size:65


## 分词，训练集/验证集划分

### 分词
这里用的是最简单的分词-字符级标记器（character-level tokenizer），创建两个字典用于存储映射关系。第一个字典存放字符->索引下标映射；第二个字典存放索引下标->字符映射。



##### 拓展知识点
在造分词器的时候，可以权衡codebook size（词汇表）和序列长度。
可以拥有词汇量非常小的非常长的整数序列，也可以拥有词汇量非常大的短整数序列。
？？？
1. 词汇量非常小的非常长的整数序列：

这指的是使用一个较小的词汇表，但允许序列（即文本中的单词或标记序列）有较长的长度。在这种情况下，每个词或标记可能由一个较大的整数来表示，因为词汇表中的项较少，所以可以使用较大的数值范围。
这种方法可能适用于某些特定的应用场景，比如当文本数据具有高度的重复性，或者当模型需要处理非常长的文本序列时。
2. 词汇量非常大的短整数序列：

这指的是使用一个较大的词汇表，但限制序列的长度。在这种情况下，每个词或标记由一个较小的整数来表示，因为词汇表中的项较多，所以每个项的表示范围较小。
这种方法可能更适用于处理多样化的文本数据，因为它能够捕捉到更多的词汇细节，但同时也需要考虑到模型的内存和计算效率，因为较大的词汇表可能会增加模型的复杂度。
--------------
个人理解

如果词汇表很大的话，那么（极端情况下）每个词/字符/字节都会有对应的索引下标。这样会造成一句话的序列可能会很长，所以一般当文本很短的时候，我们可以用大词汇表，小序列长度的方式，增加对词汇细微差别的理解。例如：社交媒体文本通常包含大量的俚语、表情符号和个性化词汇。在这种情况下，可能需要一个较大的词汇表来捕捉这些多样化的表达方式，但由于内存和计算资源的限制，序列长度可能需要被限制。

如果词汇表很小的话，序列长度很长的话。往往是用于一些重复率比较高的长文本。例如：生物信息学，
在处理基因序列数据时，序列长度可以非常长，但使用的“词汇”（即核苷酸）只有四种（A、T、C、G）。这里，序列长度是关键，而词汇量非常小。

--------------
Q:如果在特定场景应用了不恰当的分词策略，会造成什么影响？

Q:对话大模型在实际应用中会根据用户输入的字符长度来动态选择分词策略吗？

In [4]:
###Tokenizers
# create a mapping from characters to integers
str2idx = {ch: i for i, ch in enumerate(chars)}
idx2str = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [str2idx[c] for c in s]
decode = lambda l: ''.join(idx2str[c] for c in l)

In [5]:
print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


在实践中，类似的分词策略例如：Google利用SentencePiece，即一种子词类型的标记器。该标记器没有对整个单词进行编码，也没有对单个字符进行编码。它是一个子字单元级别。

OpenAI用了一个称为TickToken的库，它使用字节对编码标记器

In [6]:
#将整个数据集进行编码
import torch
data = torch.tensor(encode(text),dtype=torch.long)#这两个参数是什么意思？#这个函数在干嘛？
print(data.shape, data.dtype)
print(data[:1000])


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/apple/Library/r-miniconda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/apple/Library/r-miniconda/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/apple/Library/r-miniconda/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/Users/apple/Library/r-miniconda/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launc

torch.Size([1115394]) torch.int64
tensor([ 0, 18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43,
        44, 53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52,
        63,  1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,
         1, 57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39,
        49,  6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15,
        47, 58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50,
        50,  1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1,
        58, 53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51,
        47, 57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43,
        42,  8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57,
        58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1,
        63, 53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56,
      

### 划分训练集/验证集

In [7]:
n = int(0.9*len(data))
train_data = data[:n]#训练集
val_data = data[n:]#验证集

In [8]:
print(len(train_data))
print(len(val_data))

1003854
111540


## 数据加载器：批量数据块

当我们训练Transformer时，我们不会将所有的语料一次性喂给模型。

我们从训练集中随机抽取一个训练块进行训练，这些块的长度称为block_size

In [9]:
#假设block_size为8，第一个block为
block_size=8
train_data[:block_size+1]

tensor([ 0, 18, 47, 56, 57, 58,  1, 15, 47])

上面的代码中，之所以block_size要+1是因为：当你从训练集中抽取一块数据时，比如上述的例子中有9个字符，其实包含了8个示例。每个例子如下： 

为什么这里Karpathy提到输入的是Transformer的张量的时间维度

In [10]:
x = train_data[:block_size]#前block_size的元素
y = train_data[1:block_size+1]#这里为什么是从1开始，为什么要到
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 18
when input is tensor([ 0, 18]) the target: 47
when input is tensor([ 0, 18, 47]) the target: 56
when input is tensor([ 0, 18, 47, 56]) the target: 57
when input is tensor([ 0, 18, 47, 56, 57]) the target: 58
when input is tensor([ 0, 18, 47, 56, 57, 58]) the target: 1
when input is tensor([ 0, 18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([ 0, 18, 47, 56, 57, 58,  1, 15]) the target: 47


##### Batch size
每次我们要将它们输入Transformer时，我们都会有许多批次的多个文本块，它们都堆叠在一个张量(tensor)中，这样做的原因是为了提高效率，以便利用GPU的并行计算能力来并行处理数据。这些块是完全独立处理的，彼此之间不通信。

Every time we're going to feed them into a transformer, we're going to have many batches of multiple chunks of text that are all stacked up in a single tensor.

所以单个tensor包含了多个batch（批次），每个批次包含了多个chunks

In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for prediction?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split =='train' else val_data#输入训练数据/验证数据
    ix = torch.randint(len(data) - block_size, (batch_size,))#通过随机偏移量来选择不同的数据块
    #生成偏移量的逻辑：在0-(len(data)-block_size)的范围内，随机生成batch_size个
    x = torch.stack([data[i:i+block_size] for i in ix])#把这些随机抽取的序列进行拼接成batch
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
# print('input:')
# print(xb.shape)
# print(xb)
# print('target:')
# print(yb.shape)
# print(yb)
# 
# print('---------------')
# 
# for b in range(batch_size):#batch dimension
#     for t in range(block_size):#time dimension
#         context=xb[b,:t+1]
#         target=yb[b,t]
#         print(f"when input is {context.tolist()} the target: {target}")

## 最简单的基线：二元语法模型，损失，生成


In [12]:
vocab_size

65

In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        #token embedding table -> 标记嵌入表,例如上述tensor中[[0,24,43,...],[...]..];
        # 值为24的元素将在嵌入表中找到第24行，然后呢...?
        #nn.Embedding -> 一个非常薄的包装器，基本上是一个形状为vocab_size*vocab_size的张量
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):#这是一个封装起来的强制函数
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) #（B,T,C),在本例中,B(batch)批次为4，T(time)为8，C(chanel)为vocabSize即65
        if targets is None:
            loss = None
        else:   
            #利用负对数似然损失(negative log likelihood loss)衡量损失或预测质量，它也在PyTorch中以交叉熵的名称实现
            #但是因为Pytorch对于cross_engropy的入参格式有要求，所以这里需要变换下维度
            B,T,C = logits.shape
            logits = logits.view(B*T, C)#将原先的4*8*65的三维数组扁平化至32*65
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits,loss
        
        #但是这里没有调用logits函数啊，怎么计算的呢
    
        #如果无视Batch的话，每个矩阵是一个8*65的矩阵。好像有点感觉，8代表时间维度（为什么叫做时间维度？），65代表
        #总之得到的结果是每个senquece对于自己下一个词的预测分数？
        #然后现在需要一个方法来衡量损失
    
    def generate(self, idx, max_new_tokens):#generate函数的主要目的是利用训练好的模型进行文本生成
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)#为什么这里会调用
            # focus only on the last time step
            logits = logits[:, -1,:]# becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #(B,C)
            # sample form the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
            
        
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)


print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=100)[0].tolist()))

#print(out.shape)
#out[0].shape
        
        
        
        
        
        

torch.Size([32, 65])
tensor(4.7390, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


上述评估结果为4.73，但是-In(1/65)结果并不在这个区间内

 ## 训练模型

In [14]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [15]:
batch_size=32
for steps in range(10000):
    #sample a batch of data
    xb, yb = get_batch('train')
    
    ###经典的训练循环
    # evalulate the loss
    logits, loss = m(xb, yb)
    #将所有梯度归零->why?
    optimizer.zero_grad(set_to_none=True)
    loss.backward()#获取所有参数的梯度
    optimizer.step()#使用这些梯度来更新我们的参数
    #print(loss.item())
print(loss.item())#该优化过程非常不稳定

2.5532305240631104


In [16]:
print(decode(m.generate(idx = torch.zeros((1,1),dtype=torch.long),max_new_tokens=400)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henouratucenonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht anjx?

DUThinqunt.

LaZAnde.
athave l.
KEONH:
ARThanco be y,-hedarwnoddy scar t tridesar, wnl'shenou


### 版本1:用for循环平均过去的上下文，聚合的最弱形式

In [17]:
torch.manual_seed(1337)
B,T,C=4,8,2 #batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [18]:
#xbow: bag of words,词袋是人们在对事物进行平均时使用的术语
xbow = torch.zeros(B,T,C)
for b in range(B):#batch dimension（单次遍历：一句话）
    for t in range(T):#time dimension(时间步，单次遍历：一个词)
        xprev = x[b,:t+1]#(t,c) 为什么要加1->因为需要确保当前时间步的特征也被包括在内
        xbow[b,t] = torch.mean(xprev, 0)

In [19]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [20]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

xbow（词袋）结果是计算了每个当前时间步以及其历史的聚合特征，聚合方式是平均，虽然简单但是损失了大量空间特征

### 版本2:使用矩阵乘法

1. 矩阵乘法的特点：C(3,2)=A(3,3)*B(3,2)
2. 对于C矩阵中，第一行第一列元素为矩阵A的第一行和矩阵B第一列的点积
3. 利用torch.tril()函数，可以只返回矩阵的下角部，上角部设置为0 -> 模拟masked attention
4. 构造一个形状为time dim*time dim的数组->和步骤1中的矩阵A性质一样，这个矩阵包含了权重信息
5. x（B*T,C）->步骤1中的矩阵B
6. xbow2 = wei @ x

    做矩阵乘法：(T,T) @ (B,T,C)
   
    两个矩阵形状不同，PyTorch会自动补全第一个矩阵，将其形状变换为(B,T,C)

    得到的矩阵形状为（B,T,C）


上述步骤通过批量矩阵乘法(batch matrix multiply)，实现了加权聚合

In [21]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)#keepdim:是否平均
xbow2 = wei @ x # (T,T) @ (B,T,C)

In [22]:
torch.allclose(xbow2,xbow)#证明两个构造矩阵是相似的

True

In [23]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

### 版本3:添加Softmax

1. 权重初始为“0”而不是“1”的原因：增加亲和力，亲和力可以理解成注意力权重
2. 掩码因果关系：通过将当前时间步之后的元素'-inf'，表示我们不会从这些token中聚合任何东西
3. 为什么F.softmax()函数中的dim参数为-1 ---->每一列的权重被归一化,这通常用于处理列向量，例如在某些注意力机制中，每列代表一个时间步的权重
4. 若dim设置为1->每一行的权重被归一化


In [24]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))#掩码因果关系
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow,xbow3)

True

In [25]:
wei = F.softmax(wei, dim=1)
wei[0][0]

tensor(0.2797)

## 自注意力

自注意力解决的问题：
对于一个时间步元素，希望其他元素以基于数据依赖的方式涌向自己

**自注意力机制：**

每个节点或每个位置的每个标记都会发出两个向量：Query（查询向量）和Key（键）。查询向量意义：我在寻找什么；键向量意义：我包含什么。

在序列中获得这些标记之间的亲和力的方式基本上是在键和查询之间进行点积，所以我的查询与所有其他标记的所有键进行点积，这个点积现在变成了重量(weight)。如果某两个标记之间是一致的，它们将以非常高的量相互作用。因此，我将更多地关注这个标记，而不是序列中的其他标记。

感性理解q.k,v：

每个x可以理解成当前标记的私人信息。query是当前标记寻找的信息，key是当前标记/节点包含的信息。如果其他标记对当前标记感兴趣的话，当前标记提供的信息是value。这个比喻类似于电影分类的比喻；观众有感兴趣的电影类型（动作片，query），电影院中有不同类型的电影（爱情片，惊悚片，动作片...key），具体的电影有（北京遇上西雅图，闪灵，霍元甲...value）。


整个机制的数学表达式：
假设输入序列长度为 \( T \)，特征维度为 \( D \)。

查询 \( Q \)、键 \( K \) 和值 \( V \) 的线性变换：
$$
Q = W^Q \cdot x + b^Q
$$
输入 \( x \) 经过查询权重矩阵 \( W^Q \) 和偏置 \( b^Q \)。

$$
K = W^K \cdot x + b^K
$$
输入 \( x \) 经过键权重矩阵 \( W^K \) 和偏置 \( b^K \)。

$$
V = W^V \cdot x + b^V
$$
输入 \( x \) 经过值权重矩阵 \( W^V \) 和偏置 \( b^V \)。

计算注意力得分（缩放点积）：
$$
\text{Attention}_\text{Scores} = \frac{Q \cdot K^T}{\sqrt{D_k}}
$$

应用softmax函数归一化：
$$
\text{Attention}_\text{Weights} = \text{softmax}(\text{Attention}_\text{Scores})
$$

计算加权值：
$$
\text{Weighted}_\text{Values} = V \odot \text{Attention}_\text{Weights}
$$

求和得到最终的注意力输出：
$$
\text{Attention}_\text{Output} = \sum_{t=1}^{T} \text{Weighted}_\text{Values}_t
$$


In [26]:
torch.manual_seed(1337)
B,T,C = 4,8,32#batch, time, channels
x = torch.randn(B,T,C)

# single head self-attention
head_size=16#有什么用
key=nn.Linear(C,head_size, bias=False)
query=nn.Linear(C,head_size, bias=False)
value=nn.Linear(C,head_size, bias=False)

k=key(x)
q=query(x)
wei = q @ k.transpose(-2, -1) #(B,T,16) @ (B,16,T) ----> (B,T,T)
#为什么要转置key矩阵-
# >为了确保执行点积时，query和key向量能在维度上对齐
# ->输出维度：
# 转置键矩阵会影响点积操作的输出维度。例如，如果查询矩阵 Q 的形状是 (B, T_q, D_k)，键矩阵 K 的形状是 (B, T_k, D_k)，其中 B 是批次大小，T_q 和 T_k 分别是查询序列和键序列的长度，D_k 是键的维度。转置 K 后，K 的形状变为 (B, D_k, T_k)。这样，Q 和 K^T 进行矩阵乘法得到的注意力得分矩阵 Attention_Scores 形状将是 (B, T_q, T_k)，表示每个查询时间步对于每个键时间步的得分。


###内部机制
tril = torch.tril(torch.ones(T,T))#构造上角部掩码矩阵
wei = wei.masked_fill(tril==0, float('-inf'))#因果掩码(Encoder只需要注释掉这行，即允许当前时间步的token同时看到上下文)
wei = F.softmax(wei, dim=-1)#通过softmax概率函数，将权重归一化，使得每个列向量权重相加为1
###

v = value(x)#v是我们聚合的元素，或者我们聚合的向量，而不是原始的x；聚合成(B, T, head_size)
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [27]:
out_test = wei @ x
out_test.shape

torch.Size([4, 8, 32])

Notes:
- Attention is a communication mechanism. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.（注意力是一种通信机制。可以将其视为有向图中的节点相互观察，并根据指向它们的所有节点以数据依赖的权重尽心加权求和来聚合信息。）


- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens（注意力不包含空间信息，所以我们需要加入位置编码）


- Each example across batch dimension is of course processed completely independently and never "talk" to each other（每个batch的数据都是独立处理的，不与其他batch进行通信。）


- In an "encoder" attention block just delete the single line that does making with *trill*, allowing all tokens to communicate. Here is called a "decoder" attention block because it has triangular masking, and is usually in autoregressive setting（编码器就是删除掉因果掩码的decoder，即允许当前时间步的token和上下文所有的token之间进行通信）


- "self-attention" just means that the keys and values are produced from the same sources as queries. In "cross-attention", the queries get produced from x, but the keys and values come from some other, external sources（.e.g. an encoder module）（自注意力机制意味着key, value, query都来自于x；而交叉注意力机制中，x只生成query；而key和value都来自于外部，例如encoder） 

### “缩放”自注意力。为什么除以sqrt(head_size)构建Transformer

当权重的值过于大或者过于小的话，Softmax函数的生成值会开始锐化，甚至收敛到One-Hot Vectors。见下面例子

In [28]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [29]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1)

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

## Multi-Head Attention

前馈层的实现比较简单，只是将单注意力变成多头

## Transformer块的前馈层

1. 什么是前馈层？
在Transformer架构中，前馈层（Feed-Forward Neural Network，FFN）是位于多头自注意力机制之后的一组线性变换，用于进一步处理数据。Transformer模型的每个编码器（Encoder）和解码器（Decoder）层中都包含前馈层。

前馈层的基本结构：
第一层线性变换：输入数据首先通过一个线性层，通常是一个全连接的神经网络层，将数据从原始维度映射到一个新的维度。

非线性激活函数：在第一层线性变换之后，通常会应用一个非线性激活函数，如ReLU（Rectified Linear Unit）。

第二层线性变换：然后，数据通过另一个线性层，将数据从激活函数的输出维度映射回原始维度。

前馈层的作用：
在利用注意力机制收集了数据后，前馈层允许每个注意力头思考收集到的信息（抽象捏）

（没懂）

## 残差连接

也叫做skip connections（跳过连接），有时也称为残差连接（residual connections）

残差连接有什么用？为什么能优化模型？梯度为什么
将输入直接添加到网络的后一个后续层的输出上，从而允许网络学习残差函数。

残差连接的关键特点：

直接连接：在残差连接中，输入通过一个恒等映射（通常是1x1的卷积或简单的相加操作）直接连接到网络的深层。


简化学习：残差连接简化了网络的学习任务，因为网络可以学习输入和输出之间的残差（即差异），而不仅仅是输出。


避免梯度消失：在深层网络中，梯度可能会随着层的增加而逐渐消失。残差连接通过直接连接帮助梯度流动，从而缓解了这个问题。



## Dropout

一种正则化技术，可以在神经网络前向-后项传递时随机关闭掉一些神经元（把它们的权重调整为0），主要用来防止过拟合

dropout添加位置：
1. 前馈层末尾

2. 多头扩展末尾

3. 计算完亲和力和softmax之后（注意力得分）

#1:42:54

